In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

In [7]:
url = 'https://raw.githubusercontent.com/DavGev/OMDS_project/master/data.txt'
data = pd.read_csv(url)

In [8]:
X = data.loc[data.Y.isin(['D', 'G']), data.columns != 'Y']
y = data.loc[data.Y.isin(['D', 'G']), 'Y']
y = pd.get_dummies(y)

# PART 1 - MLP

- $ E(ω;π) = - \frac{1}{P} ∑_{i=1}^P {[y_i \ln(p_i) + (1 - y_i) \ln(1 - p_i)]} + ρ \|ω\|^2 $
- $ ρ = 10^{−4} $
- $ S(v)_j = \frac{e^{v_j}}{∑_{h=1}^n e^{v_h}} $
- The activation function $g(t) := tanh(t)$

### Hyperparameters
- the number H of hidden layers (max. 4) (only for question 1)
- the number of neurons N of the hidden layers
- the spread $σ > 0$ in the activation function $g$ ($g$ is available in Python with $σ = 1$: `numpy.tanh`)

### Tasks
- Write a program which implements the regularized training error function $E(v,w,b)$
- **Question 1. (grade up to 20)** Use an optimization algorithm from `scipy.optimize` that uses the gradient to determine the parameters $v_j ,w_{ji}, b_j$ which minimize the error.
- **Question 2. (grade up to 10)** Develop an RBF neural network trained by implementing the decomposition method studied in class.

| Ex | H | N | $σ$ | $ρ$ | Optimization | Message | Init train error | Final train error | Final  test error | f\grad evaluations | Time |
| -|-|-|-|-|-|-|-|-|-|-|-|
| Q1 Full MLP |
| Q2 RBF |


\* optimization: with parameters (optimality accuracy, max number of iterations etc)

\* message: in output (successful optimization or others, number of iterations, number of function/gradient evaluations, starting/final value of the objective function, starting/final accuracy etc)

In [9]:
class Softmax:
    def __init__(self):
        self.name = 'Softmax'

    def __call__(self, x):
        return np.exp(x) / np.exp(x).sum(axis=1)[:, None]

    def grad(self, s_x):
        I = np.diag(np.ones(s_x.shape[1]))
        return s_x[:,None,:] * (I[None,:,:] - s_x[:,:,None])


class Tanh:
    def __init__(self, sigma):
        self.name = 'Tanh'
        self.sigma = sigma

    def __call__(self, x):
        return np.tanh(x / self.sigma)

    def grad(self, th_x):
        return 1 / self.sigma * (1 - th_x ** 2)


class Linear:
    def __init__(self):
        self.name = 'Linear'

    def __call__(self, x):
        return x

    def grad(self, x):
        return 1



In [10]:
class MLPLayer():
    def __init__(self, input_size, output_size, activation):
        self.w = np.random.random((output_size, input_size)) * 0.01
        self.w /= (self.w ** 2).sum() ** 0.5
        self.activation = activation

        self.input = None
        self.output = None
        self.grad_w = None
        self.grad_input = None


    def Forward(self, input):
        self.input = np.insert(input, 0, 1, axis=-1)
        sum = self.input @ self.w.T
        self.output = self.activation(sum)


    def Backward(self, grad_output):
        if self.activation.name == 'Softmax':
            grad_sum = self.activation.grad(self.output) * grad_output[:,:,None]
            grad_sum = grad_sum.sum(axis=1)
        else:
            grad_sum = self.activation.grad(self.output) * grad_output
        self.grad_w = grad_sum.T @ self.input
        self.grad_input = grad_sum @ self.w[:,1:]


In [11]:
class MLP():
    def __init__(self, N, sigma=10):
        '''
        N: array of numbers of neurons in the input layer,
        each hiden layer and the output layer

        For example if our data is 10 dimentional, we need two hidden layers
        with 5 neurons, and we have 2 classes, than N = [10, 5, 5, 2]
        '''
        self.rho = 1e-4
        self.layers = [
            MLPLayer(
                input_size = N[i] + 1,
                output_size = N[i+1],
                activation = Tanh(sigma)
            ) for i in range(len(N) - 1)
        ]
        self.layers[-1].activation = Softmax()
        self.accuracy = None


    def assign_w(self, w):
        '''
        w: 1d array of all the weights
        '''
        start = 0
        end = 0
        for layer in self.layers:
            end += layer.w.size
            layer.w = w[start : end].reshape(layer.w.shape)
            start = end


    def get_flat(self, what):
        if what == 'w':
            return np.concatenate([layer.w.flatten() for layer in self.layers])
        if what == 'grad_w':
            return np.concatenate([layer.grad_w.flatten() for layer in self.layers])


    def predict(self, X):
        ipnut = np.asarray(X)
        for layer in self.layers:
            layer.Forward(ipnut)
            ipnut = layer.output
        return ipnut


    def error(self, X, y):
        y = np.asarray(y)
        p = self.predict(X)
        error = - (y * np.log(p)).sum(axis=1).mean()
        error += self.rho * (self.get_flat('w') ** 2).sum()
        return error


    def gradient(self, X, y):
        y = np.asarray(y)
        p = self.predict(X)
        grad_output = - (y / p) / X.shape[0]
        for layer in self.layers[::-1]:
            layer.Backward(grad_output)
            grad_output = layer.grad_input

        grad = self.get_flat('grad_w')
        grad += 2 * self.rho * self.get_flat('w')
        return grad


    def fit(self, X, y, method='BFGS'):

        def fun(w):
            self.assign_w(w)
            return self.error(X, y)

        def jac(w):
            self.assign_w(w)
            return self.gradient(X, y)

        w0 = self.get_flat('w')
        message = minimize(fun=fun, jac=jac, x0=w0, method=method)
        self.assign_w(message.x)
        return message


    def score(self, X, y, threshold=0.5):
        p = self.predict(X)
        self.accuracy = ((p > 0.5) == y).mean(axis=0)[0]
        return self.accuracy


In [17]:
model = MLP([16, 4, 2])
model.fit(X, y)
model.score(X, y)

1.0